## Below is sentiment analysis code provided by TH

In [1]:
# from nltk.tokenize import word_tokenize # or use some other tokenizer
# from nltk import NaiveBayesClassifier

# train = [('I love this sandwich.', 'pos'),
#          ('This is an amazing place!', 'pos'),
#          ('I feel very good about these beers.', 'pos'),
#          ('This is my best work.', 'pos'),
#          ("What an awesome view", 'pos'),
#          ('I do not like this restaurant', 'neg'),
#          ('I am tired of this stuff.', 'neg'),
#          ("I can't deal with this", 'neg'),
#          ('He is my sworn enemy!', 'neg'),
#          ('My boss is horrible.', 'neg') ]

# test = [('The beer was good.', 'pos'),
#         ('I do not enjoy my job', 'neg'),
#         ("I ain't feeling dandy today.", 'neg'),
#         ("I feel amazing!", 'pos'),
#         ('Gary is a friend of mine.', 'pos'),
#         ("I can't believe I'm doing this.", 'neg') ]


# def transformfeature(sentence,mydict):
#     sent_features={}
#     for w in word_tokenize(sentence):
#         sent_features[w.lower()]=w.lower() in mydict
#The order is different fromom mine, where the words from the large dict are checked against the words from the doc
    
#     return sent_features

# #classifier = nltk.NaiveBayesClassifier.train(train)

# mydict=set(word.lower() for passage in train for word in word_tokenize(passage[0]))

# #print(mydict)
# train_feature= [(transformfeature(x[0],mydict), x[1]) for x in train]
# test_feature= [(transformfeature(x[0],mydict), x[1]) for x in test]

# classifier=NaiveBayesClassifier.train(train_feature)
# print("Accuracy is:", classify.accuracy(classifier,test_feature))

### Take a look over the data

In [2]:
import pandas as pd

#Load and take a look
df=pd.read_csv("clean_nus_sms.csv")

df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48598 entries, 0 to 48597
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  48598 non-null  int64 
 1   id          48598 non-null  int64 
 2   Message     48595 non-null  object
 3   length      48598 non-null  object
 4   country     48598 non-null  object
 5   Date        48598 non-null  object
dtypes: int64(2), object(4)
memory usage: 2.2+ MB


In [3]:
#Remove empty lines
df=df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48595 entries, 0 to 48597
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  48595 non-null  int64 
 1   id          48595 non-null  int64 
 2   Message     48595 non-null  object
 3   length      48595 non-null  object
 4   country     48595 non-null  object
 5   Date        48595 non-null  object
dtypes: int64(2), object(4)
memory usage: 2.6+ MB


### Remove noise (punctuation,url,html,lower case, stop words)
#### I often forgot punctuation

In [4]:
#http://sg.briefcase.yahoo.com/
import re
from nltk.corpus import stopwords


#CA:     url_pattern = re.compile(r'https?://\S+|www\.\S+')
def remove_url(text):
    url=re.compile(r'http\://([a-z]+\.)+')
    cleaned=url.sub(r' ',text)
    #CA:return url_pattern.sub(r'', text)
    #Note that only compiled objects can be used that way (if "url" is a string,it can't)
    #That is, "re" stands for the compiled object, namely a regex object
    return cleaned

#CA:df["Message_clean"] = df["Message_clean"].apply(lambda text: remove_urls(text))
df['Message']=df['Message'].apply(remove_url)
# df['message_clean']=df['Message'].apply(lambda x:remove_url(x))
df['message_clean']=df['Message'].apply(remove_url)

# df['message_clean']=remove_url(str(df['Message']))
# df['message_clean'].loc[1304]
# df['message_clean'].loc[1304]


def remove_html(text):
    html_tag=re.compile(r'<.*?>')
    return html_tag.sub(r' ',text)

df['message_clean']=df['message_clean'].apply(lambda x:remove_html(x))

#lower case
df['message_clean']=df['message_clean'].str.lower()
df['country']=df['country'].str.lower()

def remove_punctuation(text):
    punctuation_pattern=re.compile(r'[^\w\s]')
    #Note that the "^" here means "not", instead of "initial"
    return punctuation_pattern.sub(r' ',text)

#Remove punctuation
df['message_clean']=df['message_clean'].apply(lambda x: remove_punctuation(x))

# #Remove stop words

# ",".join(stopwords.words('chinese'))
# #CA decides that it's safter to leave stop words in the text, so I'll follow suit.

C:\Users\USER\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
df['message_clean'].to_csv('message_clean_after_the_removals.csv')

### Tokenization


In [6]:
# !pip install --user -U nltk

In [7]:
import nltk
from nltk.tokenize import word_tokenize


# nltk.download('punkt')
#Q I'm not sure what it is 
#Q Also, I thought I have to do either load/download or import, not both
#Q Moreover, errors occurred downloading"""


df['tokenized_message']=df['message_clean'].apply(lambda x:word_tokenize(x))
df['tokenized_message'][1]
print(type(df['tokenized_message'][1]))

<class 'list'>


In [8]:
df['tokenized_message'].to_csv('cleaned_and_tokenized_message.csv')

## Exploratory data analysis

In [9]:
df.head()

#Usually, EDA involves looking at the relatioship between the main variable and the message
#It seems that the main variable is "message_clean"
#And the column whose relationship with message_clean is most worth noting is "country"

df['country'].value_counts()

#SG should be replaced with Singapore

df['country']=df['country'].replace('SG','Singapore')#No need to convert to .str
df['country']=df['country'].replace('my','malaysia')
#united kingdom as uk, united states as usa, srilanka as sri lanka
df['country']=df['country'].replace('uk','united kingdom')
df['country']=df['country'].replace('usa','united states')
df['country']=df['country'].replace('srilanka','sri lanka')
df['country'].value_counts()

#Although I found the majority of the countries have too few tweets to either make comparisons or classify the emotions involved
#I'll stick with CA's analysis in order to practice Pandas and become more sensitive to data types

singapore              22013
sg                      9804
india                   7085
united states           5680
sri lanka               1047
malaysia                 767
pakistan                 751
unknown                  602
canada                   198
bangladesh               126
china                    107
philippines               67
indonesia                 48
united kingdom            40
nepal                     39
hungary                   28
serbia                    22
kenya                     20
ghana                     18
italia                    10
trinidad and tobago       10
lebanon                   10
turkey                    10
nigeria                   10
new zealand               10
slovenia                  10
macedonia                 10
morocco                    9
australia                  9
romania                    9
jamaica                    8
barbados                   8
spain                      5
france                     5
Name: country,

### Sentiment analysis starts here

In [10]:
# CA: Download twitter data and sentiment analysis model 
#I'm not sure which of the following downloads involve sentiment analysis

nltk.download('twitter_samples')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

#What does the final "True" mean

[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [11]:
#Take the labeled data from twitter,separate the positve from the negative
#Q I am not familar with these downloading conventions.
#May have to find the document by directly asking a question like "does NLTK provide sentiment dataset"

from nltk.corpus import twitter_samples

# Instantiate positive tweets
positive_tweets = twitter_samples.strings('positive_tweets.json')
# Instantiate negative tweets
negative_tweets = twitter_samples.strings('negative_tweets.json')

print(f"len(positive_tweets):\n{len(positive_tweets)}")
print(f"len(negative_tweets):\n{len(positive_tweets)}")
#The NLTK Twitter corpus appears to be balanced and suitable for ML

#What does a tweet look like
print(positive_tweets[0])
print(negative_tweets[0])
print(type(positive_tweets))

len(positive_tweets):
5000
len(negative_tweets):
5000
#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)
hopeless for tmr :(
<class 'list'>


## Again, preprocess the downloaded data from nltk

In [12]:
#remove non-words/punctuation
cleaned_pos_tweets=[re.sub(r'[^\w\s]','',tweet) for tweet in positive_tweets]
#Sample: x = re.sub("\s", "9", txt)
cleaned_neg_tweets=[re.sub(r'[^\w\s]','',tweet) for tweet in negative_tweets]

#remove url
cleaned_pos_tweets=[remove_url(tweet) for tweet in positive_tweets]
cleaned_neg_tweets=[remove_url(tweet) for tweet in negative_tweets]

#remove html
cleaned_pos_tweets=[remove_html(tweet) for tweet in positive_tweets]
cleaned_neg_tweets=[remove_html(tweet) for tweet in negative_tweets]

#lower case
cleaned_pos_tweets=[tweet.lower() for tweet in positive_tweets]
cleaned_neg_tweets=[tweet.lower() for tweet in negative_tweets]

print(cleaned_pos_tweets[:3])

['#followfriday @france_inte @pkuchly57 @milipol_paris for being top engaged members in my community this week :)', '@lamb2ja hey james! how odd :/ please call our contact centre on 02392441234 and we will be able to assist you :) many thanks!', '@despiteofficial we had a listen last night :) as you bleed is an amazing track. when are you in scotland?!']


In [13]:
#Tokenize the 5000 positive/negative strings (tweets)
"""CA's way:
positive_tweets_tokenized=twitter_samples.tokenize('positive_tweets.json')
positive_tweets_tokenized=twitter_samples.tokenize('positive_tweets.json')
"""
pos_tokens=[word_tokenize(tweet) for tweet in cleaned_pos_tweets]
neg_tokens=[word_tokenize(tweet) for tweet in cleaned_neg_tweets]
pos_tokens[0]

['#',
 'followfriday',
 '@',
 'france_inte',
 '@',
 'pkuchly57',
 '@',
 'milipol_paris',
 'for',
 'being',
 'top',
 'engaged',
 'members',
 'in',
 'my',
 'community',
 'this',
 'week',
 ':',
 ')']

### NLTK-based sentiment features dict that label docs (in terms of whether the entry in the doc)

In [14]:
#Generate a token list and select the most frequent 10,000
pos_tokens_with_c=[(item,'positive') for item in pos_tokens]
neg_tokens_with_c=[(item,'negative') for item in neg_tokens]

tweets_with_c=pos_tokens_with_c+neg_tokens_with_c



print(f'tweets_with_c:\n{tweets_with_c}')


token_list=[token for sublist in sublists_as_elements for token in sublist]
#flat list of all tokens

#CA: all_tweet_tokens = [token for sublist in list_of_all_tweets for token in sublist]
# print(token_list[:6])



# tokens_dist=nltk.FreqDist(token_list)
"""
ChatGPT says "tokens_dist=nltk.FreqDist(token_list)" will count unique tokens 
#BUT this is different from how CA did it (shown below):
#CA:all_tokens = nltk.FreqDist(token for token in all_tweet_tokens)
"""
#2/5 Can't solve the "all-positive" issue, so following CA's "tokens_dist"
#Which somehow shrink the count down from 130052 to 22690

tokens_dist=nltk.FreqDist(token for token in token_list)


#inner after outer, generates a flat (i.e. one-layer list)

# most_common_words=[word for (word,freq) in tokens_dist.most_common(10000)]
most_common_words=[word for (word,freq) in tokens_dist.most_common(10000)]

tweets_with_c:
[(['#', 'followfriday', '@', 'france_inte', '@', 'pkuchly57', '@', 'milipol_paris', 'for', 'being', 'top', 'engaged', 'members', 'in', 'my', 'community', 'this', 'week', ':', ')'], 'positive'), (['@', 'lamb2ja', 'hey', 'james', '!', 'how', 'odd', ':', '/', 'please', 'call', 'our', 'contact', 'centre', 'on', '02392441234', 'and', 'we', 'will', 'be', 'able', 'to', 'assist', 'you', ':', ')', 'many', 'thanks', '!'], 'positive'), (['@', 'despiteofficial', 'we', 'had', 'a', 'listen', 'last', 'night', ':', ')', 'as', 'you', 'bleed', 'is', 'an', 'amazing', 'track', '.', 'when', 'are', 'you', 'in', 'scotland', '?', '!'], 'positive'), (['@', '97sides', 'congrats', ':', ')'], 'positive'), (['yeaaaah', 'yippppy', '!', '!', '!', 'my', 'accnt', 'verified', 'rqst', 'has', 'succeed', 'got', 'a', 'blue', 'tick', 'mark', 'on', 'my', 'fb', 'profile', ':', ')', 'in', '15', 'days'], 'positive'), (['@', 'bhaktisbanter', '@', 'pallaviruhail', 'this', 'one', 'is', 'irresistible', ':', ')', '#',

NameError: name 'sublists_as_elements' is not defined

### Below is the error-prone function

In [ ]:
def document_sentiment_dict(doc,most_common_words):
    sentiment_features={}
    doc_tokens=set(doc)
    #set() turns the list into a set, supposedly speeding up the processing.
    for token in most_common_words:#Loop over nltk's 10,000
        sentiment_features[token]=(token in doc_tokens)
        #Note how entries are added to the dict
    return sentiment_features


### Find out the positive/negative probability for each word. First, create a feature set

In [ ]:
import random

pos_tweets_labeled=[(tweet,"positive") for tweet in cleaned_pos_tweets]
neg_tweets_labeled=[(tweet,"negative") for tweet in cleaned_neg_tweets] 



tweets_labeled=pos_tweets_labeled+neg_tweets_labeled

print(tweets_labeled[:3])
# print('\n')

tweets_labeled_tokenized=[]
for d,c in tweets_labeled[:3]:
    d=d.split()
    new=(d,c)
    tweets_labeled_tokenized.append(new)

print(f'tweets_labeled_tokenized[:3]:{tweets_labeled_tokenized[:3]}')
print('\n')

##Initialize a random set but make that choice permanent by setting a seed value
# random.seed(42)#1/31 change the number because previously all the predictions are positive

random.seed(44)

#Q CA shuffles the order but I thought "random" suffices
random.shuffle(tweets_labeled)


feature_list=[((document_sentiment_dict(d,most_common_words)),c) for\
             (d,c) in tweets_labeled]
#2/6 the latter part is probably incorrect: "d" here should be tokenized but 'tweets_labeled' are not

# feature_list=[(document_sentiment_dict(d,most_common_words),c) for (d,c) \
#               in tweets_with_c]
    

#2/2 CA: feature_set = [(document_features(d, sentiment_features),c) for (d,c) in all_labeled_tweets]
print(f"前三條feature:{feature_list[:3]}")#A list with all the elements as well as the whole tweet labelled

train_set,test_set=feature_list[:5000],feature_list[5000:]

### Use Naive Bayes

In [ ]:
from nltk import classify
from nltk import NaiveBayesClassifier

# classifier=nltk.NaiveBayesClassifier()
#Below is from CA:
classifier = nltk.NaiveBayesClassifier.train(train_set)
#Important: it appears that nltk.NaiveBayesClassifier takes the second argument as answers because
#The second argument is either "positive" or "negative"

# CA: Evaluate the model with classify() function
print("Accuracy is:", classify.accuracy(classifier, test_set))
#Why is mone much lower?(mine:0.4952,CA: 0.9963333333333333)
#I thought this is because I have done the wrong preprocessing. 
"""
CA has done:
remove_html: checked
remove_url:checked
re.sub(r'[^\w\s]',' '): I did but CA didn't. BUT MY ACCURACY REMAIN THE SAME BEFORE AND AFTER THE REPLACEMENT.
"""

#CA: Show the 20 most informative features for prediction
print(classifier.show_most_informative_features(50))#QQ: Since these are as likely positive as negative,why are they informative? 

### Finally, use the model to classify NSU messages (though it did badly in my case)
#### CA's equivalent: "Now I will evaluate all the messages in the NUS SMS Corpus."

In [ ]:

"""Sample:

feature_list=[((document_sentiment_dict(d,most_common_words)),c) for\
             (d,c) in tweets_labeled]
train_set,test_set=[feature_list[:5000],feature_list[5000:]]

classifier=NaiveBayesClassifier.train(train_set)
print("Accuracy is:", classify.accuracy(classifier,test_set))
"""

"""
BELOW IS CAs:
"""

sentiments=[]

for message in df['tokenized_message']:
    sentiments.append(str(classifier.classify(dict([(token,True) for token in message]))))
    #Q Why always true?
    ###BIGGER ISSUE: why always positive? 1/30
    ###CA: Positive    47336; Negative     1259

df['sentiment']=sentiments

print(df['tokenized_message'][:3])

df['sentiment'].value_counts()

print(df['sentiment'][:3])

### So far, I still haven't figured out what the True/False in these features lists mean

In [ ]:
## Below is to list all the functions I've defined

In [ ]:
""" You can use the globals() 
and locals() functions to access the global and local namespaces, respectively,"""

# List all the functions defined in the global namespace (not within a specific cell)
import types
functions = [obj for obj in globals().values() if isinstance(obj, types.FunctionType)]
function_names = [func.__name__ for func in functions]

# Print the list of function names
print(function_names)

# List all the variables defined in the global namespace (not within a specific cell)
variables = [var for var in globals().keys() if not var.startswith('_')]
variable_names = [var for var in variables if not callable(globals()[var])]

print(variable_names)

In [ ]:
#To ease future machine learning, I'm converting the True/False values to 0 and 1.

df['sentiments_0_or_1']=df['sentiment'].apply(lambda x: 1 if x=='positive' else 0)
df['sentiments_0_or_1'][:3]

In [ ]:
df_sent=df.groupby(['country']).sentiments_0_or_1.mean()
df_sent[:3]